In [1]:
import json
import polars as pl
from pathlib import Path

base = Path("/Users/davidenicoli/Local_Workspace/Datasets/ARPA/VDA/")

In [2]:
with open(base / "stations.json", "rt") as f:
    stations = json.load(f)["stations"]

In [3]:
cf_meta = pl.from_records(stations).select(pl.col("marker_name").alias("name"), pl.col("marker_lon").alias("lon"), pl.col("marker_lat").alias("lat"))

In [7]:
import pandas as pd
db_meta = pl.from_pandas(pd.read_excel("/Users/davidenicoli/Local_Workspace/TesiMag/external/correzioni/VDA_edit.xlsx")[["name"]])

In [10]:
db_meta.join(cf_meta, on="name", how="left").to_pandas().to_excel("/Users/davidenicoli/Local_Workspace/TesiMag/external/correzioni/VDA_edit_locs.xlsx", index=False) 

In [22]:
import geopandas as gpd
import fiona

md = cf_meta.to_pandas()
gdf = gpd.GeoDataFrame(cf_meta.to_pandas(), geometry=gpd.points_from_xy(md.lon, md.lat))
fiona.supported_drivers["KML"] = "raw"
gdf.to_file("/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Valle D'Aosta/cf.kml", driver="KML")